In [ ]:
%load_ext watermark


In [ ]:
import os

import alifedata_phyloinformatics_convert as apc
from hstrat._auxiliary_lib import (
    alifestd_collapse_unifurcations,
    alifestd_downsample_tips_asexual,
    alifestd_downsample_tips_clade_asexual,
    alifestd_join_roots,
    alifestd_mark_leaves,
    alifestd_mark_origin_time_delta_asexual,
    alifestd_prune_extinct_lineages_asexual,
    alifestd_to_working_format,
    alifestd_try_add_ancestor_list_col,
)
import iplotx as ipx
from matplotlib import pyplot as plt
import polars as pl
from teeplot import teeplot as tp
from scipy import stats as scipy_stats


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2026-02-12-billion-tip-phylos"
)
teeplot_subdir


## Prep Data


In [ ]:
phylo1_df = alifestd_try_add_ancestor_list_col(
    alifestd_mark_origin_time_delta_asexual(
        alifestd_to_working_format(
            alifestd_join_roots(
                pl.read_parquet("https://osf.io/download/485tx").to_pandas(),
            ),
        ),
    ),
)
phylo1_df


In [ ]:
phylo2_df = alifestd_try_add_ancestor_list_col(
    alifestd_mark_origin_time_delta_asexual(
        alifestd_to_working_format(
            alifestd_join_roots(
                pl.read_parquet("https://osf.io/download/vkzdw").to_pandas(),
            ),
        ),
    )
)
phylo2_df


## Origin Time Stats


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["origin_time"].mean(), phylo2_df["origin_time"].mean()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["origin_time"].median(), phylo2_df["origin_time"].median()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["origin_time"].max(), phylo2_df["origin_time"].max()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["origin_time"].min(), phylo2_df["origin_time"].min()),
)
v1, v2, v1 / v2


In [ ]:
scipy_stats.mannwhitneyu(
    phylo1_df["origin_time"].dropna(),
    phylo2_df["origin_time"].dropna(),
)


## Origin Time Delta Stats


In [ ]:
phylo1_df["ot_delta"] = phylo1_df["origin_time_delta"]
phylo2_df["ot_delta"] = phylo2_df["origin_time_delta"]


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["ot_delta"].mean(), phylo2_df["ot_delta"].mean()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["ot_delta"].median(), phylo2_df["ot_delta"].median()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["ot_delta"].max(), phylo2_df["ot_delta"].max()),
)
v1, v2, v1 / v2


In [ ]:
v1, v2 = map(
    float,
    (phylo1_df["ot_delta"].min(), phylo2_df["ot_delta"].min()),
)
v1, v2


In [ ]:
scipy_stats.mannwhitneyu(
    phylo1_df["origin_time_delta"].dropna(),
    phylo2_df["origin_time_delta"].dropna(),
)


## Whole-tree Sample


In [ ]:
tree1 = apc.RosettaTree(
    alifestd_downsample_tips_asexual(phylo1_df, 250, seed=1),
).as_dendropy
tree2 = apc.RosettaTree(
    alifestd_downsample_tips_asexual(phylo2_df, 250, seed=1),
).as_dendropy


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(4, 2),
    teeplot_outattrs={"layout": "vertical", "sample": "down"},
    teeplot_subdir=teeplot_subdir,
) as teed:
    fig, (ax1, ax2) = teed
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="vertical",
        edge_linewidth=1.5,
        margins=0.0,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        layout="vertical",
        edge_linewidth=1.5,
        margins=0.0,
    )


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(6, 1.5),
    gridspec_kw={
        "wspace": 0.0,
        "hspace": 0.0,
    },
    teeplot_subdir=teeplot_subdir,
    teeplot_outattrs={"layout": "vertical", "sample": "down"},
) as teed:
    fig, (ax1, ax2) = teed

    style = [
        "tree",
        {
            "layout": {
                "start": -180,
                "span": 180,
            },
        },
    ]
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        angle=270,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    fig.tight_layout()


## Clade Sample


In [ ]:
tree1 = apc.RosettaTree(
    alifestd_downsample_tips_clade_asexual(phylo1_df, 4_000, seed=2),
).as_dendropy
tree2 = apc.RosettaTree(
    alifestd_downsample_tips_clade_asexual(phylo2_df, 4_000, seed=2),
).as_dendropy


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(6, 2),
    gridspec_kw={
        "wspace": 0.0,
        "hspace": 0.0,
    },
    teeplot_subdir=teeplot_subdir,
    teeplot_outattrs={"layout": "vertical", "sample": "clade"},
) as teed:
    fig, (ax1, ax2) = teed
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="vertical",
        edge_linewidth=0.5,
        margins=-0.02,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        layout="vertical",
        edge_linewidth=0.5,
        margins=-0.02,
    )
    fig.tight_layout()


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(6, 1.5),
    gridspec_kw={
        "wspace": 0.0,
        "hspace": 0.0,
    },
    teeplot_subdir=teeplot_subdir,
    teeplot_outattrs={"layout": "vertical", "sample": "clade"},
) as teed:
    fig, (ax1, ax2) = teed

    style = [
        "tree",
        {
            "layout": {
                "start": -180,
                "span": 180,
            },
        },
    ]
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        angle=270,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    fig.tight_layout()


## Canopy Sample


In [ ]:
phylo1_df = alifestd_mark_leaves(phylo1_df)
phylo2_df = alifestd_mark_leaves(phylo2_df)


In [ ]:
n = 4_000
phylo1_df["extant"] = False
phylo1_df.loc[phylo1_df["is_leaf"], "extant"] = (
    phylo1_df.loc[phylo1_df["is_leaf"], "origin_time"].rank(
        ascending=False,
        method="first",
    )
    <= n
)
phylo2_df["extant"] = False
phylo2_df.loc[phylo2_df["is_leaf"], "extant"] = (
    phylo2_df.loc[phylo2_df["is_leaf"], "origin_time"].rank(
        ascending=False,
        method="first",
    )
    <= n
)

phylo1_df["extant"].sum(), phylo2_df["extant"].sum()


In [ ]:
tree1 = apc.RosettaTree(
    # alifestd_mark_origin_time_delta_asexual(
    #     alifestd_collapse_unifurcations(
    alifestd_prune_extinct_lineages_asexual(phylo1_df),
    #     ),
    # ),
).as_dendropy
tree2 = apc.RosettaTree(
    # alifestd_mark_origin_time_delta_asexual(
    #     alifestd_collapse_unifurcations(
    alifestd_prune_extinct_lineages_asexual(phylo2_df),
    # ),
    # ),
).as_dendropy


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(6, 2),
    gridspec_kw={
        "wspace": 0.0,
        "hspace": 0.0,
    },
    teeplot_subdir=teeplot_subdir,
    teeplot_outattrs={"layout": "vertical", "sample": "canopy"},
) as teed:
    fig, (ax1, ax2) = teed
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="vertical",
        edge_linewidth=0.5,
        margins=-0.02,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        layout="vertical",
        edge_linewidth=0.5,
        margins=-0.02,
    )
    fig.tight_layout()


In [ ]:
tree1.ladderize()
tree2.ladderize()

with tp.teed(
    plt.subplots,
    1,
    2,
    figsize=(6, 1.5),
    gridspec_kw={
        "wspace": 0.0,
        "hspace": 0.0,
    },
    teeplot_subdir=teeplot_subdir,
    teeplot_outattrs={"layout": "vertical", "sample": "canopy"},
) as teed:
    fig, (ax1, ax2) = teed

    style = [
        "tree",
        {
            "layout": {
                "start": -180,
                "span": 180,
            },
        },
    ]
    ipx.plotting.tree(
        tree1,
        ax=ax1,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    ipx.plotting.tree(
        tree2,
        ax=ax2,
        angle=270,
        layout="radial",
        # layout_orientation="right",
        edge_linewidth=0.5,
        margins=-0.02,
        style=style,
    )
    fig.tight_layout()
